In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Input & Output State

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

model = ChatOpenAI()

In [ ]:
from typing import TypedDict


class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [ ]:
def call_model(state: ChatMessages):
    question = state["question"]
    llm_calls = state.get("llm_calls", 0)
    state["llm_calls"] = llm_calls + 1
    print("LLM_CALLS:", state["llm_calls"])
    response = model.invoke(input=question)
    state["answer"] = response.content
    return state

In [ ]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:
graph.invoke(input={"question": "Whats the highest mountain in the world?"})

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


"""
This section introduces an important concept: separating state into different categories:

InputState: What goes into the graph
PrivateState: Internal data used by the graph but not exposed
OutputState: What comes out of the graph
OverallState: Combines all three for internal use
"""

class InputState(TypedDict):
    question: str


class PrivateState(TypedDict):
    llm_calls: int


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, PrivateState, OutputState):
    pass

In [ ]:
"""
Now when compiling the graph, we specify:

The complete state type (OverallState)
The input state type (InputState)
The output state type (OutputState)

This ensures that the graph only accepts the input fields and only returns the output fields, even though internally it uses all the fields.
"""
workflow = StateGraph(OverallState, input=InputState, output=OutputState)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:
graph.invoke({"question": "Whats the highest mountain in the world?"})

### Add runtime configuration

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain.schema import SystemMessage, HumanMessage

"""
This new version of call_model adds a config parameter that allows for runtime configuration. In this case, it uses the config to get a language preference and creates a system message telling the model to respond in that language.
"""
def call_model(state: OverallState, config: RunnableConfig):
    language = config["configurable"].get("language", "English")
    system_message_content = f"Respond in {language}"
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}

In [ ]:
"""
The graph setup is the same, but the node function now can use the config parameter.
"""
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:

"""
Now when invoking the graph, we pass a configuration object that specifies the desired language. This allows the same graph to behave differently based on the configuration.
"""
config = {"configurable": {"language": "Spanish"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

In [ ]:
config = {"configurable": {"language": "German"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

"""
Why Use Runtime Configuration?
You're correct that both approaches will achieve the same result of having the LLM respond in the desired language. However, there are a few reasons why the runtime configuration approach might be preferred:

Separation of Concerns: The configuration approach separates the "what" (the graph structure) from the "how" (specific parameters). Your graph structure stays the same, but its behavior can be modified.
External Control: With runtime configuration, the language choice can be made by the caller of the graph rather than being hardcoded or requiring changes to the graph itself.
Reusability: The same graph can be reused for different languages without having to rebuild it. You just invoke it with different configurations.
Consistency with Other Parameters: In more complex applications, you might have many configurable parameters. Using the config object provides a consistent way to handle all of them.
Thread Management: In a real application, the config object often carries other important information like thread IDs for conversation management, as seen in other notebooks.
"""